# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
R. Hviding  ->  R. Hviding  |  ['R. Hviding']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']


Arxiv has 68 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.16596


extracting tarball to tmp_2503.16596...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
R. Hviding  ->  R. Hviding  |  ['R. Hviding']
F. Walter  ->  F. Walter  |  ['F. Walter']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.16844


extracting tarball to tmp_2503.16844... done.
Retrieving document from  https://arxiv.org/e-print/2503.16946


extracting tarball to tmp_2503.16946...

 done.


Found 162 bibliographic references in tmp_2503.16946/main.bbl.
Issues with the citations
syntax error in line 249: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16596-b31b1b.svg)](https://arxiv.org/abs/2503.16596) | **A "Black Hole Star" Reveals the Remarkable Gas-Enshrouded Hearts of the Little Red Dots**  |
|| R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>R. Hviding</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *Submitted. NIRSpec prism spectra featured in this work from JWST program GO-5224 ("Mirage or Miracle", PIs: Oesch & Naidu) are publicly available at this https URL . See De Graaff et al. in today's arXiv posting for a z=3.5 BH*. Comments greatly appreciated and warmly welcomed!*|
|**Abstract**|            The physical processes that led to the formation of billion solar mass black holes within the first 700 million years of cosmic time remain a puzzle. Several theoretical scenarios have been proposed to seed and rapidly grow black holes, but direct observations of these mechanisms remain elusive. Here we present a source 660 million years after the Big Bang that displays singular properties: among the largest Hydrogen Balmer breaks reported at any redshift, broad multi-peaked H$\beta$ emission, and Balmer line absorption in multiple transitions. We model this source as a "black hole star" (BH*) where the Balmer break and absorption features are a result of extremely dense, turbulent gas forming a dust-free "atmosphere" around a supermassive black hole. This source may provide evidence of an early black hole embedded in dense gas -- a theoretical configuration proposed to rapidly grow black holes via super-Eddington accretion. Radiation from the BH* appears to dominate almost all observed light, leaving limited room for contribution from its host galaxy. We demonstrate that the recently discovered "Little Red Dots" (LRDs) with perplexing spectral energy distributions can be explained as BH*s embedded in relatively brighter host galaxies. This source provides evidence that black hole masses in the LRDs may be over-estimated by orders of magnitude -- the BH* is effectively dust-free contrary to the steep dust corrections applied while modeling LRDs, and the physics that gives rise to the complex line shapes and luminosities may deviate from assumptions underlying standard scaling relations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16600-b31b1b.svg)](https://arxiv.org/abs/2503.16600) | **A remarkable Ruby: Absorption in dense gas, rather than evolved stars, drives the extreme Balmer break of a Little Red Dot at $z=3.5$**  |
|| <mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *19 pages, 9 figures; submitted to A&A. See Naidu et al. for a z=8 Balmer Cliff!*|
|**Abstract**|            The origin of the rest-optical emission of compact, red, high-redshift sources known as `little red dots' (LRDs) poses a major puzzle. If interpreted as starlight, it would imply that LRDs would constitute the densest stellar systems in the Universe. However, alternative models suggest active galactic nuclei (AGN) may instead power the rest-optical continuum. Here, we present JWST/NIRSpec, NIRCam and MIRI observations from the RUBIES and PRIMER programs of The Cliff: a bright LRD at $z=3.55$ with an exceptional Balmer break, twice as strong as that of any high-redshift source previously observed. The spectra also reveal broad Hydrogen (H$\alpha\ \rm FWHM\sim1500$km/s) and He I emission, but no significant metal lines. We demonstrate that massive evolved stellar populations cannot explain the observed spectrum, even when considering unusually steep and strong dust attenuation, or reasonable variations in the initial mass function. Moreover, the formally best-fit stellar mass and compact size ($M_*\sim10^{10.5}\,M_\odot,\ r_{e}\sim40\,$pc) would imply densities at which near-monthly stellar collisions might lead to significant X-ray emission. We argue that the Balmer break, emission lines, and H$\alpha$ absorption line are instead most plausibly explained by a `black hole star' (BH*) scenario, in which dense gas surrounds a powerful ionising source. In contrast to recently proposed BH* models of dust-reddened AGN, we show that spectral fits in the rest UV to near-infrared favour an intrinsically redder continuum over strong dust reddening. This may point to a super-Eddington accreting massive black hole or, possibly, the presence of (super)massive stars in a nuclear star cluster. The Cliff is the clearest evidence to date that at least some LRDs are not ultra-dense, massive galaxies, and are instead powered by a central ionising source embedded in dense, absorbing gas.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16608-b31b1b.svg)](https://arxiv.org/abs/2503.16608) | **Limits on the atmospheric metallicity and aerosols of the sub-Neptune GJ 3090 b from high-resolution CRIRES+ spectroscopy**  |
|| L. T. Parker, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *17 pages, 11 figures. Accepted for publication in MNRAS*|
|**Abstract**|            The sub-Neptune planets have no solar system analogues, and their low bulk densities suggest thick atmospheres containing degenerate quantities of volatiles and H/He, surrounding cores of unknown sizes. Measurements of their atmospheric composition can help break these degeneracies, but many previous studies at low spectral resolution have largely been hindered by clouds or hazes, returning muted spectra. Here, we present the first comprehensive study of a short-period sub-Neptune using ground-based, high-resolution spectroscopy, which is sensitive to the cores of spectral lines that can extend above potential high altitude aerosol layers. We observe four CRIRES+ $\textit{K}$-band transits of the warm sub-Neptune GJ 3090 b (T$_{\text{eq}}$ = 693$\pm$18 K) which orbits an M2V host star. Despite the high quality data and sensitivity to CH$_4$, H$_2$O, NH$_3$, and H$_2$S, we detect no molecular species. Injection-recovery tests are consistent with two degenerate scenarios. First, GJ 3090 b may host a highly metal-enriched atmosphere with $>\,$150 Z$_{\odot}$ and mean molecular weight $>\,$7.1 g mol$^{-1}$, representing a volatile dominated envelope with a H/He mass fraction $x_{\text{H/He}} < 33\%$, and an unconstrained aerosol layer. Second, the data are consistent with a high altitude cloud or haze layer at pressures $<\,$3.3$\times$10$^{-5}~$bar, for any metallicity. GJ 3090 b joins the growing evidence to suggest that high metallicity atmospheres and high altitude aerosol layers are common within the warm (500$~<T_{\text{eq}}<~$800 K) sub-Neptune population. We discuss the observational challenges posed by the M-dwarf host star, and suggest observing strategies for transmission spectroscopy of challenging targets around M-dwarfs for existing and ELT instrumentation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16609-b31b1b.svg)](https://arxiv.org/abs/2503.16609) | **First Measurements of Black Hole Accretion and Radio-jet Timescales in a Young Quasar at the Edge of Reionization**  |
|| S. Rojas-Ruiz, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *20 pages, 9 figures, submitted to ApJ*|
|**Abstract**|            We present the first study dedicated to measuring the timescales for black hole accretion and jet launch in a quasar at the edge of Reionization, PSO J352.4034-15.3373 at z = 5.832 $\pm$ 0.001. Previous work presented evidence of the strong radio synchrotron emission from the jet affecting the host galaxy dust-dominated continuum emission at $\nu_{\rm rest}=683$ GHz ($\nu_{\rm obs}=100$ GHz), implying a break in the synchrotron spectrum. In this work, we present quasi-simultaneous observations at 1.5\, GHz - 42\,GHz with the Karl G. Jansky Very Large Array (VLA), and derive a frequency break at $\nu^{\rm break}_{\rm rest} = 196.46$ GHz ($\nu^{\rm break}_{\rm obs} = 28.76$ GHz). Modeling these observations, we calculate the jet spectral aging from the cooling of electrons to be $t_{\mathrm{spec}}\sim 580$ yr. From this measurement, we approximate the dynamical age $t_{\mathrm{dyn}}$ to be $\sim2,000$ yr, implying a recent jet ejection. We compare the jet timescale to the quasar's lifetime ($t_{\mathrm{Q}}$) that indicates the duration of the latest black hole accretion event and is derived from the proximity zone size in the rest-UV/optical spectrum. However, a ghostly Damped Ly$\alpha$ (DLA) system affects this measurement yielding an upper limit of $t_{\mathrm{Q}} \lesssim 10^4$ yr, consistent with the jet lifetime and indicative of a young quasar. This suggests that the triggering of a UV-bright quasar phase may occur within comparable timescales as the launch of a relativistic radio jet. Therefore, we may be witnessing an early stage of black hole and jet interactions in a quasar during the first gigayear of the universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16637-b31b1b.svg)](https://arxiv.org/abs/2503.16637) | **Euclid: Star clusters in IC 342, NGC 2403, and Holmberg II**  |
|| S. S. Larsen, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *Submitted to A&A following internal review*|
|**Abstract**|            We examine the star cluster populations in the three nearby galaxies IC 342, NGC 2403, and Holmberg II, observed as part of the Euclid Early Release Observations programme. Our main focus is on old globular clusters (GCs), for which the wide field-of-view and excellent image quality of Euclid offer substantial advantages over previous work. For IC 342 this is the first study of stellar clusters other than its nuclear cluster. After selection based on size and magnitude criteria, followed by visual inspection, we identify 111 old (> 1 Gyr) GC candidates in IC 342, 50 in NGC 2403 (of which 15 were previously known), and 7 in Holmberg II. In addition, a number of younger and/or intermediate-age candidates are identified. The colour distributions of GC candidates in the two larger galaxies show hints of bimodality with peaks at IE-HE = 0.36 and 0.79 (IC 342) and IE-HE = 0.36 and 0.80 (NGC 2403), corresponding to metallicities of [Fe/H]=-1.5 and [Fe/H]=-0.5, similar to those of the metal-poor and metal-rich GC subpopulations in the Milky Way. The luminosity functions of our GC candidates exhibit an excess of relatively faint objects, relative to a canonical, approximately Gaussian GC luminosity function (GCLF). The excess objects may be similar to those previously identified in other galaxies. The specific frequency of classical old GCs in IC 342, as determined based on the brighter half of the GCLF, appears to be unusually low with SN=0.2-0.3. The combined luminosity function of young and intermediate-age clusters in all three galaxies is consistent with a power-law distribution, dN/dL ~ L^(-2.3+/-0.1) and the total numbers of young clusters brighter than M(IE)=-8 in NGC 2403 and Holmberg II are comparable with those found in their Local Group counterparts, that is, M33 and the Small Magellanic Cloud, respectively.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16946-b31b1b.svg)](https://arxiv.org/abs/2503.16946) | **Observational constraints on the origin of the elements. IX. 3D NLTE abundances of metals in the context of Galactic Chemical Evolution Models and 4MOST**  |
|| <mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark>, <mark>R. Hoppe</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *13 pages + 16 pages appendix, 10 figures + 17 figures appendix, accepted to MNRAS*|
|**Abstract**|            Historically, various methods have been employed to understand the origin of the elements, including observations of elemental abundances which have been compared to Galactic Chemical Evolution (GCE) models. It is also well known that 1D Local Thermodynamic Equilibrium (LTE) measurements fail to accurately capture elemental abundances. Non-LTE (NLTE) effects may play a significant role, and neglecting them leads to erroneous implications in galaxy modelling. In this paper, we calculate 3D NLTE abundances of seven key iron-peak and neutron-capture elements (Mn, Co, Ni, Sr, Y, Ba, Eu) based on carefully assembled 1D LTE literature measurements, and investigate their impact within the context of the OMEGA+ GCE model. Our findings reveal that 3D NLTE abundances are significantly higher for iron-peak elements at [Fe/H]< -3, with (for the first time ever) [Ni/Fe] and (confirming previous studies) [Co/Fe] on average reaching 0.6-0.8 dex, and [Mn/Fe] reaching -0.1 dex, which current 1D core-collapse supernova (CCSN) models cannot explain. We also observe a slightly higher production of neutron-capture elements at low metallicities, with 3D NLTE abundances of Eu being higher by +0.2 dex at [Fe/H]= -3. 3D effects are most significant for iron-peak elements in the very metal-poor regime, with average differences between 3D NLTE and 1D NLTE {reaching} up to 0.15 dex. Thus, ignoring 3D NLTE effects introduces significant biases, so including {them} should be considered whenever possible.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16690-b31b1b.svg)](https://arxiv.org/abs/2503.16690) | **Making the unmodulated pyramid wavefront sensor smart II. First on-sky demonstration of extreme adaptive optics with deep learning**  |
|| R. Landman, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-24*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Pyramid wavefront sensors (PWFSs) are the preferred choice for current and future extreme adaptive optics (XAO) systems. Almost all instruments use the PWFS in its modulated form to mitigate its limited linearity range. However, this modulation comes at the cost of a reduction in sensitivity, a blindness to petal-piston modes, and a limit to the sensor's ability to operate at high speeds. Therefore, there is strong interest to use the PWFS without modulation, which can be enabled with nonlinear reconstructors. Here, we present the first on-sky demonstration of XAO with an unmodulated PWFS using a nonlinear reconstructor based on convolutional neural networks. We discuss the real-time implementation on the Magellan Adaptive Optics eXtreme (MagAO-X) instrument using the optimized TensorRT framework and show that inference is fast enough to run the control loop at >2 kHz frequencies. Our on-sky results demonstrate a successful closed-loop operation using a model calibrated with internal source data that delivers stable and robust correction under varying conditions. Performance analysis reveals that our smart PWFS achieves nearly the same Strehl ratio as the highly optimized modulated PWFS under favorable conditions on bright stars. Notably, we observe an improvement in performance on a fainter star under the influence of strong winds. These findings confirm the feasibility of using the PWFS in its unmodulated form and highlight its potential for next-generation instruments. Future efforts will focus on achieving even higher control loop frequencies (>3 kHz), optimizing the calibration procedures, and testing its performance on fainter stars, where more gain is expected for the unmodulated PWFS compared to its modulated counterpart.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16844-b31b1b.svg)](https://arxiv.org/abs/2503.16844) | **Upper limits on the gamma-ray emission from the microquasar V4641 Sgr**  |
|| Z. Zhao, <mark>J. Li</mark>, D. F. Torres |
|*Appeared on*| *2025-03-24*|
|*Comments*| *10 pages, 6 figures, accepted by ApJ*|
|**Abstract**|            Following a recent detection of TeV radiation by the Large High Altitude Air Shower Observatory (LHAASO) and the High-Altitude Water Cherenkov Observatory (HAWC), coincident with the direction of the microquasar V4641 Sgr, we search for possible GeV emission from this source. We explored the morphology and temporal features of the source as well as two nearby unassociated point sources which could be a part of extended structure of V4641 Sgr, and compared results with corresponding X-ray and TeV emissions. The 95% confidence level upper limits for the flux from the source, assuming both point and extended source models were 5.38$\times$ 10$^{-13}$ erg cm$^{-2}$ s$^{-1}$ and 1.12$\times$ 10$^{-12}$ erg cm$^{-2}$ s$^{-1}$, respectively. Additionally, no correlation between gamma-ray light curve and X-ray outbursts was observed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.16596/./CLOUDYmodel_v6.png', 'tmp_2503.16596/./simultaneous_fit.png', 'tmp_2503.16596/./symmetry_v3.png']
copying  tmp_2503.16596/./CLOUDYmodel_v6.png to _build/html/
copying  tmp_2503.16596/./simultaneous_fit.png to _build/html/
copying  tmp_2503.16596/./symmetry_v3.png to _build/html/
exported in  _build/html/2503.16596.md
    + _build/html/tmp_2503.16596/./CLOUDYmodel_v6.png
    + _build/html/tmp_2503.16596/./simultaneous_fit.png
    + _build/html/tmp_2503.16596/./symmetry_v3.png
found figures ['tmp_2503.16600/./figures/comp_z7_lrds_lum.png', 'tmp_2503.16600/./figures/miles_fig.png', 'tmp_2503.16600/./figures/sed_fits.png']
copying  tmp_2503.16600/./figures/comp_z7_lrds_lum.png to _build/html/
copying  tmp_2503.16600/./figures/miles_fig.png to _build/html/
copying  tmp_2503.16600/./figures/sed_fits.png to _build/html/
exported in  _build/html/2503.16600.md
    + _build/html/tmp_2503.16600/./figures/comp_z7_lrds_lum.png
    + _build/html/tmp_2503.16600/./figu

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pasa}{PASA}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\thefigure}{Extended Data \arabic{figure}}$
$\newcommand{\thetable}{Extended Data \arabic{table}}$</div>



<div id="title">

# A "Black Hole Star" Reveals the Remarkable Gas-Enshrouded Hearts of the Little Red Dots

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16596-b31b1b.svg)](https://arxiv.org/abs/2503.16596)<mark>Appeared on: 2025-03-24</mark> -  _Submitted. NIRSpec prism spectra featured in this work from JWST program GO-5224 ("Mirage or Miracle", PIs: Oesch & Naidu) are publicly available at this https URL . See De Graaff et al. in today's arXiv posting for a z=3.5 BH*. Comments greatly appreciated and warmly welcomed!_

</div>
<div id="authors">

R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>R. Hviding</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The physical processes that led to the formation of billion solar mass black holes within the first 700 million years of cosmic time remain a puzzle Banados18, Yang20Poniuaena, Wang21quasar . Several theoretical scenarios have been proposed to seed and rapidly grow black holes Woods19, Smith19SMBH, Inayoshi20, Volonteri21 , but direct observations of these mechanisms remain elusive. Here we present a source 660 million years after the Big Bang that displays singular properties: among the largest Hydrogen Balmer breaks reported at any redshift, broad multi-peaked H $\beta$ emission, and Balmer line absorption in multiple transitions. We model this source as a  “black hole star" (BH*) where the Balmer break and absorption features are a result of extremely dense, turbulent gas forming a dust-free "atmosphere" around a supermassive black hole Inayoshi24, Ji25BlackThunder . This source may provide evidence of an early black hole embedded in dense gas -- a theoretical configuration proposed to rapidly grow black holes via super-Eddington accretion AlexanderNatarajan14, Begelman08quasistars, Volonteri10, Schleicher13, CoughlinBegelman24 . Radiation from the BH* appears to dominate almost all observed light, leaving limited room for contribution from its host galaxy. We demonstrate that the recently discovered “Little Red Dots" (LRDs) Matthee24, Greene24, Kocevski24, Taylor24, Lin24 with perplexing spectral energy distributions Akins24, Wang24z8, Setton24b, Kokorev24, Labbe24Monster, Labbe25LRDs, Killi24 can be explained as BH*s embedded in relatively brighter host galaxies. This source provides evidence that SMBH masses in the LRDs may be over-estimated by orders of magnitude -- the BH* is effectively dust-free contrary to the steep dust corrections applied while modeling LRDs Greene24, Brooks24, Furtak24, Ji25BlackThunder , and the physics that gives rise to the complex line shapes and luminosities may deviate from assumptions underlying standard scaling relations.

</div>

<div id="div_fig1">

<img src="tmp_2503.16596/./CLOUDYmodel_v6.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Comparison against a mock spectrum of a "black hole star" model**. The schematic depicts an SMBH within a $\approx40$ AU "atmosphere" of dense gas -- the continuum is produced in hot regions close to the SMBH whereas absorption, scattering, and further emission occur in the dense gas atmosphere. The data (navy blue) are binned ($3\times$) to emphasize the continuum shape that our fiducial model (pink, with noise as per error spectrum) provides an excellent match to. The model is selected to reproduce the Balmer break strength and Balmer line EWs, while also matching the UV-faintness and MIRI long-wavelength data without having to invoke different mechanisms for lines and continuum. The narrow [OIII] emission and additional UV luminosity plausibly arise from the faint host galaxy, and are not captured by the BH* model (see Fig. 4). The excess flux around H$\infty$ is a \texttt{Cloudy} artifact due to modeling with a finite number of Hydrogen levelsJi25BlackThunder. (**)

</div>
<div id="div_fig2">

<img src="tmp_2503.16596/./simultaneous_fit.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Extended Data Figure 1 $|$ Simultaneous emission line fits to the prism (top) and grating (bottom) spectra.** 100 draws from the posterior are shown in orange. The consistency of features across both modes inspires confidence in their reality. For example, the detailed structure of the H$\beta$ line -- a central absorber, extremely broad wings, and absorption even in the wings -- recurs in both panels. Similarly, the existence of narrow [OIII]5008Å emission would be difficult to discern in either mode by itself, but is recovered in the joint fit. (**)

</div>
<div id="div_fig3">

<img src="tmp_2503.16596/./symmetry_v3.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Extended Data Figure 2 $|$ The remarkable symmetry of the H$\beta$ line profile points to a symmetric configuration of absorbing gas.** Here we compare the observed line profile (top, blue) to a line profile mirrored around the systemic redshift derived from [OIII](center, orange). In the bottom panel we co-add these spectra. These profiles display peaks ($\pm1000$ km s$^{-1}$, and perhaps also $\pm2400$ km s$^{-1}$) and troughs ($\pm1500$ km s$^{-1}$; see absorber locations in Table 3) at similar velocities. This potential symmetry has important implications. First, it means the recovered systemic redshift from [OIII] is robust. Furthermore, these features are highly unlikely to be the result of random absorption systems or inflows/outflows that are fortuitously aligned at the same positive and negative velocities. Instead, this alignment suggests the presence of a symmetric absorbing structure (e.g., shells of gas). (**)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16596"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\rm \mu m}$
$\newcommand{\ergs}{\rm erg s^{-1}}$
$\newcommand{\Oiii}{[O {\sc iii}]\xspace}$
$\newcommand{\Hei}{He {\sc i}\xspace}$
$\newcommand{\Oi}{O {\sc i}\xspace}$
$\newcommand{\Nii}{[N {\sc ii}]\xspace}$
$\newcommand{\Hb}{{\rm H}\beta\xspace}$
$\newcommand$
$\newcommand{\Msun}{\rm M_\odot}$
$\newcommand{\kpc}{ \rm kpc}$
$\newcommand{\kms}{ \rm km s^{-1}}$
$\newcommand{\re}{r_{\rm e,maj}}$
$\newcommand{\rec}{r_{\rm e}}$
$\newcommand{\fdm}{f_{\rm DM}}$
$\newcommand{\fbar}{f_{\rm baryon}}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdm}{M_{\rm DM}}$
$\newcommand{\Mbar}{M_{\rm baryon}}$
$\newcommand{\Mdyn}{M_{\rm dyn}}$
$\newcommand{\Mbh}{M_{\rm BH}}$
$\newcommand{\reff}{r_{\rm e}}$
$\newcommand{\Ha}{H\alpha\xspace}$
$\newcommand{\Pad}{Pa-\delta\xspace}$
$\newcommand{\Pag}{Pa-\gamma\xspace}$
$\newcommand{\AG}[1]{{\bf \color{purple}{#1} }}$
$\newcommand{\ruby}{\textit{The Cliff}\xspace}$
$\newcommand{\zspec}{z_{\rm spec}}$
$\newcommand{\zphot}{z_{\rm phot}}$
$\newcommand{\pix}{\rm pix}$
$\newcommand{\prospector}{\texttt{Prospector}\xspace}$
$\newcommand{\pysersic}{\texttt{pysersic}\xspace}$
$\newcommand{\Av}{A_{\rm V}}$
$\newcommand{\cloudy}{\texttt{Cloudy}\xspace}$
$\newcommand{\mombh}{MoM-BH*-1\xspace}$</div>



<div id="title">

# A remarkable Ruby: Absorption in dense gas, rather than evolved stars, drives the extreme Balmer break of a Little Red Dot at $z=3.5$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16600-b31b1b.svg)](https://arxiv.org/abs/2503.16600)<mark>Appeared on: 2025-03-24</mark> -  _19 pages, 9 figures; submitted to A&A. See Naidu et al. for a z=8 Balmer Cliff!_

</div>
<div id="authors">

<mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The origin of the rest-optical emission of compact, red, high-redshift sources known as `little red dots' (LRDs) poses a major puzzle. If interpreted as starlight, it would imply that LRDs would constitute the densest stellar systems in the Universe. However, alternative models suggest active galactic nuclei (AGN) may instead power the rest-optical continuum.Here, we present JWST/NIRSpec, NIRCam and MIRI observations from the RUBIES and PRIMER programs of $\ruby$ : a bright LRD at $z=3.55$ with an exceptional Balmer break, twice as strong as that of any high-redshift source previously observed. The spectra also reveal broad Hydrogen ( $\Ha$ $\rm FWHM\sim1500 \kms$ ) and $\Hei$ emission, but no significant metal lines. We demonstrate that massive evolved stellar populations cannot explain the observed spectrum, even when considering unusually steep and strong dust attenuation, or reasonable variations in the initial mass function.Moreover, the formally best-fit stellar mass and compact size ( $M_*\sim10^{10.5} \Msun, r_{\rm e}\sim40 $ pc) would imply densities at which near-monthly stellar collisions might lead to significant X-ray emission.We argue that the Balmer break, emission lines, and $\Ha$ absorption line are instead most plausibly explained by a ` $*black hole star*$ ' (BH*) scenario, in which dense gas surrounds a powerful ionising source. In contrast to recently proposed BH* models of dust-reddened AGN, we show that spectral fits in the rest UV to near-infrared favour an intrinsically redder continuum over strong dust reddening. This may point to a super-Eddington accreting massive black hole or, possibly, the presence of (super)massive stars in a nuclear star cluster. $\ruby$ is the clearest evidence to date that at least some LRDs are not ultra-dense, massive galaxies, and are instead powered by a central ionising source embedded in dense, absorbing gas.

</div>

<div id="div_fig1">

<img src="tmp_2503.16600/./figures/comp_z7_lrds_lum.png" alt="Fig9" width="100%"/>

**Figure 9. -** The NIRSpec/PRISM spectra of $\ruby$ and $\mombh$($\zspec=7.76$; Naidu et al. 2025) are remarkably similar in shape, and only differ by a factor $\sim2$ in luminosity. Top: A BH* star model of a blue incident AGN spectrum ($\alpha_X=-1.5$, $\alpha_{\rm UV}=-0.5$) with high column density of $N_{\rm H}=10^{24} $cm$^{-2}$(rescaled to the rest $[4000,4100] Å$ flux of $\ruby$ and convolved to PRISM resolution) yields an intrinsically strong Balmer break (dark green) that matches the spectrum better than any of the stellar spectra explored (Figure $\re$f{fig:miles}). Steep dust curves are required to match the shape of the spectrum blueward of $\sim4200 Å$(green), and high optical depths are needed to be able to match the curvature of the rest-optical spectrum (light green, orange). However, such strongly dust-reddened AGN models result in a severe discrepancy in the rest near-IR. Bottom: Increased absorption from dense gas ($N_{\rm H}=10^{26} $cm$^{-2}$) results in extra reddening of the spectra without the need for dust (blue line), but the Balmer break of this model does not match the spectrum. A UV-weak incident AGN spectrum ($\alpha_X=-0.5$, $\alpha_{\rm UV}=-0.1$) can produce a strong Balmer break as well as a redder rest near-IR, although small mismatches in the IR remain even after modest dust attenuation (red lines). This suggests a redder intrinsic AGN spectrum (as predicted by some super-Eddington accretion disc models), or additional component in the SED such as (super)massive stars, is needed to explain $\ruby$.  (*fig:BHstar*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16600/./figures/miles_fig.png" alt="Fig8" width="100%"/>

**Figure 8. -** Top left: Balmer break strength (from rest-frame $[3620,3720] Å$ and $[4000,4100] Å$ ranges) distribution for the standard MILES stellar library (blue), and after applying a Calzetti dust law (with $\Av=1$; dashed purple), and the steep attenuation law from the $\prospector$ galaxy-only model (solid red). The black vertical line and shaded region show the measured break strength and $1\sigma$ uncertainty of $\ruby$. Top right: Effective temperatures of all MILES stars (blue) and the stars that, after applying the steep dust law, fall within $1\sigma$ of $\ruby$(red). Bottom: A representative sample of MILES spectra with strong Balmer breaks, convolved to the PRISM resolution at $z=3.5$ and normalised at $[4000,4100] Å$. Dotted and solid lines show the stellar spectra without dust and with steep dust attenuation, respectively. In comparison to the spectrum of $\ruby$, K giants, main-sequence A stars and horizontal branch stars fall short at $<3645 Å$ and $>5000 Å$. Only massive supergiants, with steep dust attenuation, can match the strength and shape of the Balmer break (albeit with a mismatch at $>5000 Å$), and suggests that an extraordinary, top-heavy IMF would be required to explain the rest-frame optical emission of $\ruby$.  (*fig:miles*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16600/./figures/sed_fits.png" alt="Fig6" width="100%"/>

**Figure 6. -** Best-fit SED models and residuals (with respect to the PRISM spectrum) for four model variations, constructed as different mixtures of (dust-reddened) stellar population and power-law AGN model components. These AGN models do not consider the possible reddening by absorbing dense gas, which we explore in Section $\re$f{sec:cloudy}. The region around the Balmer limit is highlighted, where the mismatch between models and data is strongest. From top to bottom: the fiducial `galaxy-only' model from $\prospector$; a galaxy + AGN model that maximises the stellar contribution, fit with $\prospector$(the `maximal $M_*$' model of  ([Wang and Leja 2024]()) ); the galaxy + AGN model following [Labbe, Greene and Matthee (2024)](), but fitting only Hydrogen emission lines instead of a forest of metal lines; the galaxy + AGN model of [Labbe, Greene and Matthee (2024)](), but with an even steeper dust law (see Section $\re$f{sec:dust}). All four models favour a massive post-starburst solution, with a very steep dust attenuation law and high optical depth. However, even with such extreme dust, none of these models can produce the strong Balmer break and shape of the rest-frame optical SED, as is evident from the systematic (and significant) features in the residuals blue- and redward of the Balmer break.  (*fig:seds*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16600"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\LTP}[1]{\textcolor{red}{\textsf{LTP: #1}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Limits on the atmospheric metallicity and aerosols of the sub-Neptune GJ 3090 b from high-resolution CRIRES+ spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16608-b31b1b.svg)](https://arxiv.org/abs/2503.16608)<mark>Appeared on: 2025-03-24</mark> -  _17 pages, 11 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

L. T. Parker, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** The sub-Neptune planets have no solar system analogues, and their low bulk densities suggest thick atmospheres containing degenerate quantities of volatiles and H/He, surrounding cores of unknown sizes. Measurements of their atmospheric composition can help break these degeneracies, but many previous studies at low spectral resolution have largely been hindered by clouds or hazes, returning muted spectra. Here, we present the first comprehensive study of a short-period sub-Neptune using ground-based, high-resolution spectroscopy, which is sensitive to the cores of spectral lines that can extend above potential high altitude aerosol layers. We observe four CRIRES+ _K_ -band transits of the warm sub-Neptune GJ 3090 b (T $_{\text{eq}}$ = 693 $\pm$ 18 K) which orbits an M2V host star. Despite the high quality data and sensitivity to $CH_4$ , $H_2$ O, $NH_3$ , and $H_2$ S, we detect no molecular species. Injection-recovery tests are consistent with two degenerate scenarios. First, GJ 3090 b may host a highly metal-enriched atmosphere with > 150 Z $_{\odot}$ and mean molecular weight > 7.1 g mol $^{-1}$ , representing a volatile dominated envelope with a H/He mass fraction $x_{\text{H/He}} < 33\%$ , and an unconstrained aerosol layer. Second, the data are consistent with a high altitude cloud or haze layer at pressures < 3.3 $\times$ 10 $^{-5}$ bar, for any metallicity. GJ 3090 b joins the growing evidence to suggest that high metallicity atmospheres and high altitude aerosol layers are common within the warm (500 $< T_{\text{eq}} <$ 800 K) sub-Neptune population. We discuss the observational challenges posed by the M-dwarf host star, and suggest observing strategies for transmission spectroscopy of challenging targets around M-dwarfs for existing and ELT instrumentation.

</div>

<div id="div_fig1">

<img src="tmp_2503.16608/./data_processing_bold_ABsep.png" alt="Fig4" width="100%"/>

**Figure 4. -** The data processing stages for the four CRIRES+ transits, with transit ingress and egress marked by the white dashed horizontal lines. Panel A shows the optimally extracted and phase ordered spectra, following basic calibrations including background subtraction. Panel B shows the normalised, wavelength calibrated spectra, where any residual large scale trends have been removed through division by a smoothed spectrum. Panel C shows the spectra shifted into the stellar rest frame, where saturated telluric lines and high variance columns have been masked following bad pixel correction and division by the master spectrum constructed from the mean of all exposures. Finally, Panel D shows the spectra after sysrem is applied iteratively to remove remaining linear systematic trends in the data. These resulting cleaned spectra are subsequently cross-correlated with planetary models. This figure shows the B nod spectra extracted from the first order of the second detector during the second transit. (*fig:data_proc*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16608/./log-L_allinj_constraints.png" alt="Fig11" width="100%"/>

**Figure 11. -** Recovered significance on the planetary signal ($\Delta\sigma$) when performing all-species injection-recovery tests i.e. where a model containing spectral features from all considered molecules ($CH_4$, $H_2$O, $NH_3$, $H_2$S, $CO_2$, CO) is injected, and a template containing only a single molecule is used to recover it. The exception is the top left panel where the cross-correlation template matches the all-species injected model. Dark regions denote areas of the parameter space where the injected model is confidently recovered to $>$5$\sigma$ and thus ruled out as a plausible scenarios for the planetary atmosphere. The sensitivity of the data to $CH_4$ is largely invariant across the all vs. single-species injection methods (see Figure \ref{fig:injections_ind_inj}). Low metallicities with low cloud decks are strongly disfavoured. The molecules for which we have no constraining power in these injection-recovery tests are shown in Figure \ref{fig:injections_no_constrainign_power}.
 (*fig:injections_all_inj*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16608/./inj_noinj_comparison_singleinj_4x.png" alt="Fig5" width="100%"/>

**Figure 5. -** Left: the combined $K_{\text{p}}$--$v_{\text{sys}}$ map from the four K2166 transits of GJ 3090 b for cross-correlation with $CH_4$, $H_2$O, $NH_3$ and $H_2$S templates (1 bar cloud deck, 10 Z$_{\odot}$ metallicity). No signals are observed at the expected planetary orbital and systemic velocities at sufficient significance to support a planetary origin. Note the anti-correlation at the planetary $K_{\text{p}}$, which is offset by $\sim$5 km s$^{-1}$ from the planetary systemic velocity. This likely arises from the overlap of stellar and telluric residual noise when combining the four transits, but does not impact the recovery significance of injected models when using the log(L) mapping. Right: the recovery of the same models (1 bar cloud deck, 10 Z$_{\odot}$ metallicity) injected into the 4 transit datasets. Each of these models can be recovered at high S/N, demonstrating the constraining power of the data. (*fig:real_vs_inj*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16608"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\checktext}[1]{\textbf{\textcolor{orange}{#1}}}$
$\newcommand{\fix}[1]{\textcolor{red}{#1}}$
$\newcommand{\lsim}{\mbox{_<\atop^{\sim}}}$
$\newcommand{\gsim}{\mbox{_>\atop^{\sim}}}$
$\newcommand{\eseven}{\times10^{-17}}$
$\newcommand{\eeight}{\times10^{-18}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\han}{H\alpha+[\ion{N}{2}]}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\oiiib}{[\ion{O}{3}]\lambda5007}$
$\newcommand{\sii}{[\ion{S}{2}]}$
$\newcommand{\hi}{H {\sc i}}$
$\newcommand{\hii}{H {\sc ii}}$
$\newcommand{\hei}{He {\sc i}}$
$\newcommand{\heii}{He {\sc ii}}$
$\newcommand{\nii}{[Ni {\sc ii}}$
$\newcommand{\ciii}{C {\sc iii}]}$
$\newcommand{\ciiiS}{C {\sc iii}]}$
$\newcommand{\siiv}{Si {\sc iv}}$
$\newcommand{\civ}{C {\sc iv}}$
$\newcommand{\nv}{N {\sc v}}$
$\newcommand{\cii}{[C {\sc ii}]}$
$\newcommand{\ciiS}{C {\sc ii}}$
$\newcommand{\oiii}{[O {\sc iii}]}$
$\newcommand{\oii}{[O {\sc ii}]}$
$\newcommand{\oi}{O {\sc i}}$
$\newcommand{\mgii}{Mg {\sc ii}}$
$\newcommand{\siii}{Si {\sc ii}}$
$\newcommand{\feii}{Fe {\sc ii}}$
$\newcommand{\alii}{Al {\sc ii}}$
$\newcommand{\Mdust}{\textit{M}_{\mathrm{Dust}} }$
$\newcommand{\Lfir}{\textit{L}_{\mathrm{FIR}}}$
$\newcommand{\Ltir}{\textit{L}_{\mathrm{TIR}} }$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\es}{erg s^{-1}}$
$\newcommand{\esc}{erg s^{-1} cm^{-2}}$
$\newcommand{\esca}{erg s^{-1} cm^{-2} Å^{-1}}$
$\newcommand{\esa}{erg s^{-1} Å^{-1}}$
$\newcommand{\vband}{V_{\mathrm{F606W}}}$
$\newcommand{\ibband}{I_{\mathrm{F600LP}}}$
$\newcommand{\iband}{I_{\mathrm{F814W}}}$
$\newcommand{\ybband}{Y_{\mathrm{F098M}}}$
$\newcommand{\yband}{Y_{\mathrm{F105W}}}$
$\newcommand{\yj}{J_{\mathrm{F110W}}}$
$\newcommand{\jband}{J_{\mathrm{F125W}}}$
$\newcommand{\jh}{JH_{\mathrm{F140W}}}$
$\newcommand{\hband}{H_{\mathrm{F160W}}}$
$\newcommand{\hst}{\textit{HST}}$
$\newcommand{\jwst}{\textit{JWST}}$
$\newcommand{\spitzer}{\textit{Spitzer}}$
$\newcommand{\se}{\textit{Source Extractor}}$
$\newcommand{\rhalf}{r_{1/2}}$
$\newcommand{\reff}{R_{\mathrm{eff}}}$
$\newcommand{\Pisco}{ULAS J1342+0928}$</div>



<div id="title">

# First Measurements of Black Hole Accretion and Radio-jet Timescales in a Young Quasar at the Edge of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16609-b31b1b.svg)](https://arxiv.org/abs/2503.16609)<mark>Appeared on: 2025-03-24</mark> -  _20 pages, 9 figures, submitted to ApJ_

</div>
<div id="authors">

S. Rojas-Ruiz, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark>

</div>
<div id="abstract">

**Abstract:** We present the first study dedicated to measuring the timescales for black hole accretion and jet launch in a quasar at the edge of Reionization, PSO J352.4034--15.3373 at $z\!=\!5.832 \pm 0.001$ .Previous work presented evidence of the strong radio synchrotron emission from the jet affecting the host galaxy dust-dominated continuum emission at $\nu_{\rm rest}=683$ GHz ( $\nu_{\rm obs}=100$ GHz), implying a break in the synchrotron spectrum. In this work, we present quasi-simultaneous observations at 1.5  GHz -- 42 GHz with the Karl G. Jansky Very Large Array (VLA), and derive a frequency break at $\nu^{\rm break}_{\rm rest} = 196.46$ GHz ( $\nu^{\rm break}_{\rm obs} = 28.76$ GHz). Modeling these observations, we calculate the jet spectral aging from the cooling of electrons to be $t_{\mathrm{spec}}\sim 580$ yr. From this measurement, we approximate the dynamical age $t_{\mathrm{dyn}}$ to be $\sim2,000$ yr, implying a recent jet ejection. We compare the jet timescale to the quasar's lifetime ( $t_{\mathrm{Q}}$ ) that indicates the duration of the latest black hole accretion event and is derived from the proximity zone size in the rest-UV/optical spectrum. However, a ghostly Damped Ly $\alpha$ (DLA) system affects this measurement yielding an upper limit of $t_{\mathrm{Q}} \lesssim 10^4$ yr, consistent with the jet lifetime and indicative of a young quasar. This suggests that the triggering of a UV-bright quasar phase may occur within comparable timescales as the launch of a relativistic radio jet. Therefore, we may be witnessing an early stage of black hole and jet interactions in a quasar during the first gigayear of the universe.

</div>

<div id="div_fig1">

<img src="tmp_2503.16609/./R_p_estimate_Mpc.png" alt="Fig6" width="100%"/>

**Figure 6. -** Spectrum of P352--15 normalized by the predicted continuum from the PCA analysis (black) and the noise (grey). Given the strong low ionization absorber (centered at the location of the pink vertical line), the normalized flux is less than half at the boundary of the position of the quasar. The flux smoothed with a boxcar kernel of 20 Å is shown in red. Notice that despite the presence of the strong absorber, significant transmission extends from the position of the quasar (yellow vertical region) up to where the transmission and the smoothed spectrum reach the zero level (shaded in purple). This region corresponds to the measured proximity zone bounds $R_{\rm p}\sim2-2.3$ pMpc and is the conservative upper limit we use for the calculation of the quasar lifetime $t_{\mathrm{Q}}$. (*Rp*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16609/./frequency_break_curvefit_CI_model_sub.png" alt="Fig3.1" width="50%"/><img src="tmp_2503.16609/./frequency_break_curvefit_KP_model_sub.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** The mm to radio SED fit of P352--15 based on different synchrotron break models.The mm to radio SED of P352--15. The mm-observations presented here are from \citetalias{rojas-ruiz_impact_2021}, and the radio observations are from this work in all the VLA from 1.5 GHz - 42 GHz. We report the combined fit of the cold dust emission, modeled with an MBB, and a synchrotron break, modeled with the CI _(top)_ and KP _(bottom)_ models (see \S\ref{model-break}). We note that both fits are consistent with our observations, the difference relies on the assumption of relativistic electron distribution. The resulting spectral aging of the jet $t_{\mathrm{spec}}$ from both models falls within the same order of magnitude and for the rest of this work we consider both values as upper and lower estimates, correspondingly. (*sync-models*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16609/./P352_ghost_dla.png" alt="Fig1" width="100%"/>

**Figure 1. -** P352--15 spectrum (black) and different models predicting the Ly$\alpha$ emission. The model for the predicted unabsorbed spectrum is shown in blue. The DLA system is shown centered at the position of the red dotted vertical line. The continuum fully covered by the DLA is shown with a red dashed line. Given that partial flux is observed, we model the continuum prediction when partially covered by the DLA as shown in orange. Each model has the $\pm1\sigma$ and $\pm2\sigma$ uncertainties in shaded colors. (*ghost*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16609"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$: Star clusters in IC 342, NGC 2403, and Holmberg II$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16637-b31b1b.svg)](https://arxiv.org/abs/2503.16637)<mark>Appeared on: 2025-03-24</mark> -  _Submitted to A&A following internal review_

</div>
<div id="authors">

S. S. Larsen, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We examine the star cluster populations in the three nearby (3.20--3.45 Mpc) galaxies IC 342,  NGC 2403, and Holmberg II, observed as part of the $\Euclid$ Early Release Observations programme.Our main focus in this paper is on old globular clusters (GCs), for which the wide field-of-view and excellent image quality of $\Euclid$ offer substantial advantages over previous work. For IC 342, in particular, this is the first study of stellar clusters other than its nuclear cluster.After selection based on size and magnitude criteria, followed by visual inspection, we identify 111 old ( $\gtrsim1$ Gyr) GC candidates in IC 342, 50 in NGC 2403 (of which 15 were previously known), and 7 in Holmberg II.In addition, a number of younger and/or intermediate-age candidates are identified. The colour distributions of GC candidates in the two larger galaxies show hints of bimodality with peaks at $(\IE-\HE)_0 = 0.36$ and 0.79 (IC 342) and $(\IE-\HE)_0 = 0.36$ and 0.80 (NGC 2403), corresponding to metallicities of $\mathrm{[Fe/H]}\approx-1.5$ and $\mathrm{[Fe/H]}\approx-0.5$ , similar to those found for the metal-poor and metal-rich GC subpopulations in the Milky Way.The luminosity functions of our GC candidates exhibit an excess of relatively faint objects, relative to a canonical, approximately Gaussian GC luminosity function (GCLF). Although some contamination from background galaxies may be present in our samples, we argue that the excess may be partly real, particularly in IC 342 where the excess objects may be similar to those previously identified in galaxies such as M101 and NGC 6946. The specific frequency of classical old GCs in IC 342, as determined based on the brighter half of the GCLF, appears to be unusually low with $S_N\approx0.2$ --0.3.The combined luminosity function of young and intermediate-age clusters in all three galaxies is consistent with a power-law distribution, $\mathrm{d}N/\mathrm{d}L \propto L^{-2.3\pm0.1}$ . The total numbers of young clusters brighter than $M(\IE)=-8$ in NGC 2403 and Holmberg II are comparable with those found in their Local Group counterparts, that is, M33 and the Small Magellanic Cloud, respectively.

</div>

<div id="div_fig1">

<img src="tmp_2503.16637/./SSPcol_Euc_CFHT.png" alt="Fig8" width="100%"/>

**Figure 8. -** PARSEC simple stellar population model colours versus age. Top: \Euclid$\IE-\HE$. Middle:  MegaCam $g-r$. Bottom: MegaCam $u-g$. For the MegaCam filters, dashed and solid lines indicate models for the old and new MegaCam filters, respectively. The horizontal dashed lines indicate the colour criteria for selection of old GC candidates. (*fig:SSPcol_age*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16637/./sim_near_ESCC-IC342-551.png" alt="Fig3.1" width="33%"/><img src="tmp_2503.16637/./sim_near_ESCC-IC342-525.png" alt="Fig3.2" width="33%"/><img src="tmp_2503.16637/./sim_near_ESCC-IC342-396.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Top and centre panels: as Fig. \ref{fig:simGCsN2403}, but for IC 342. Bottom panel: simulated clusters with ages of 20 Myr. (*fig:simGCsI342*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16637/./ic342_yc_gc.png" alt="Fig27" width="100%"/>

**Figure 27. -** \Euclid VIS image of IC 342 with class 1--4 cluster candidates marked.  Old GC candidates are shown with red circles, intermediate-age candidates  in blue, and ambiguous candidates as open black circles. The dashed black circle indicates a radius of 10 kpc and the white/magenta dashed lines indicate the HST/ACS F435W+F606W coverage from programmes 10579 and 16002. North is up and east to the left.  (*fig:ic342cc_yc_gc*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16637"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ns}[1]{{\color{red} #1}}$
$\newcommand{\mb}[1]{{\color{brown}{\bf Maria:} #1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\vmic}{\xi_{\rm t}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\EW}{W_{\lambda}}$
$\newcommand{\mA}{{\rm mÅ}}$
$\newcommand{\Elow}{E_{\rm low}}$
$\newcommand{\Eup}{E_{\rm up}}$
$\newcommand{\SH}{S\!_{\rm H}}$
$\newcommand{\Eu}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Y}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Sr}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Ba}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Mn}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Ni}[5]{\mbox{{\rm #1} ^#2{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\CH}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Co}[5]{\mbox{\rm #1 ^{\rm #2}{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Fe}[5]{\mbox{#1 ^#2{\rm #3}^{{\rm #4}}_ }}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Observational constraints on the origin of the elements. IX. 3D NLTE abundances of metals in the context of Galactic Chemical Evolution Models and 4MOST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16946-b31b1b.svg)](https://arxiv.org/abs/2503.16946)<mark>Appeared on: 2025-03-24</mark> -  _13 pages + 16 pages appendix, 10 figures + 17 figures appendix, accepted to MNRAS_

</div>
<div id="authors">

<mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark>, <mark>R. Hoppe</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Historically, various methods have been employed to understand the origin of the elements, including observations of elemental abundances which have been compared to Galactic Chemical Evolution (GCE) models. It is also well known that 1D Local Thermodynamic Equilibrium (LTE) measurements fail to accurately capture elemental abundances. Non-LTE (NLTE) effects may play a significant role, and neglecting them leads to erroneous implications in galaxy modelling. In this paper, we calculate 3D NLTE abundances of seven key iron-peak and neutron-capture elements (Mn, Co, Ni, Sr, Y, Ba, Eu) based on carefully assembled 1D LTE literature measurements, and investigate their impact within the context of the OMEGA+ GCE model. Our findings reveal that 3D NLTE abundances are significantly higher for iron-peak elements at [ Fe/H ]  $< -3$ , with (for the first time ever) [ Ni/Fe ] and (confirming previous studies) [ Co/Fe ] on average reaching 0.6-0.8 dex, and [ Mn/Fe ] reaching $-0.1$ dex, which current 1D core-collapse supernova (CCSN) models cannot explain. We also observe a slightly higher production of neutron-capture elements at low metallicities, with 3D NLTE abundances of Eu being higher by +0.2 dex at [ Fe/H ]  $= -3$ . 3D effects are most significant for iron-peak elements in the very metal-poor regime, with average differences between 3D NLTE and 1D NLTE ${reaching}$ up to 0.15 dex. Thus, ignoring 3D NLTE effects introduces significant biases, so including ${them}$ should be considered whenever possible.

</div>

<div id="div_fig1">

<img src="tmp_2503.16946/./figures/Mn_corrections.png" alt="Fig16" width="100%"/>

**Figure 16. -** Average Mn corrections for 1D LTE abundances at [Mn/Fe] = -0.2. Corrections are plotted for 1D NLTE and 3D NLTE respectively in red squares and black dots for three line groups in each column. Each row represents a different atmosphere, top to bottom: $\teff=6000$ K, $\logg=4.5$ dex, $\vmic = 1.00$ km s$^{-1}$(main-sequence); $\teff=6500$ K, $\logg=4.0$ dex, $\vmic = 1.50$ km s$^{-1}$(turn-off); $\teff=5500$ K, $\logg=3.5$ dex, $\vmic = 1.25$ km s$^{-1}$(subgiant). $\vmic$ only refers to the value used in 1D models. A vertical shaded area represents a rough indication where a line becomes too weak to be measured reliably (EW $\leq$5mÅ). (*fig:mn_corrections*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16946/./figures/ref1_atmo_temp_1d_3d_rev1_feb25_v2.png" alt="Fig8" width="100%"/>

**Figure 8. -** {Zoomed-in temperature structure of the 1D MARCS (red line) and the 3D RHD \texttt{Stagger} model atmosphere (blue distribution) for $\teff = 6500$ K, $\logg = 4.0$ dex, and [Fe/H] $= -3$. The red cross indicates the formation height of the Ni I line at 5476 Å as a function of $\log \tau_{500}$ in 1D LTE; similarly, the formation heights in 3D NLTE are indicated by green contours. Here the formation height is defined as the location where the line core's contribution function contributes the most.} (*fig:1d_vs_3d_temperature*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16946/./figures/mn281kbc_3d_cont_func_comp_big_v6.png" alt="Fig4" width="100%"/>

**Figure 4. -** {2D slices in the x-z plane of the 3D \texttt{Stagger} model atmosphere with parameters $\teff = 6500$ K, $\logg = 4.0$ dex, and [Fe/H] = -3. Top left: vertical velocity v$_{\rm z}$ with black contours denoting $\log \tau_{500}$ values of $-3, -1, 1$, from top to bottom (contours are preserved throughout other panels). Top right: temperature structure. Middle panels: 3D LTE of the contribution function (defined in the text) in arbitrary units for Mn line at 4030.76 Å with [Mn/Fe] = 0.0 (left) and Ni line 5476.90 Å with [Ni/Fe] = 0.5 (right). Bottom panels: same as middle panels but for the 3D NLTE case.} (*fig:cont_func_3d_both*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16946"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

402  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

38  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
